In [16]:
using Pkg
Pkg.activate("./MISAEM")

  Activating project at `~/Documents/Code/misaem_julia/MISAEM`


In [17]:
using MISAEM
using Random
using Distributions
using LinearAlgebra
using Statistics

# Set a random seed for reproducibility
# Random.seed!(123)

# 2. Generate some example data
# n: number of samples, p: number of features
n = 21000
n_test = 1000
p = 5

# Create a design matrix X with some missing values (NaN)
X_complete = rand(Normal(), n, p)
X_missing = copy(X_complete)
X_missing[rand(Bool, n, p) .< 0.15] .= NaN # Introduce 15% missing values

X_missing_fit = copy(X_missing[1:(n - n_test), :])
rows_to_keep = [!all(isnan, X_missing_fit[i, :]) for i in 1:size(X_missing_fit, 1)]
X_missing_fit = X_missing_fit[rows_to_keep, :]

X_missing_test = copy(X_missing[(n - n_test + 1):end, :])


# Create a response vector y (binary)
true_coeffs = randn(p)
linear_predictor = X_complete[:, 1:p] * true_coeffs
probabilities = 1.0 ./ (1.0 .+ exp.(-linear_predictor))
y = rand.(Bernoulli.(probabilities));

y_fit = y[1:(n - n_test)]
y_fit = y_fit[rows_to_keep]
y_test = y[(n - n_test + 1):end];

In [18]:
# 3. Instantiate the MISAEM model
# You can adjust hyperparameters here if needed
model = MISAEM.SAEMLogisticRegression(
    maxruns = 1000,
    random_state = 42,
    var_cal=true,
    ll_obs_cal=false
);

In [19]:
MISAEM.fit!(model, X_missing_fit, y_fit) # Warm-up
@time MISAEM.fit!(model, X_missing_fit, y_fit);

SAEM Iterations:   9%|███▏                              |  ETA: 0:00:06


...converged after 108 iterations.


SAEM Iterations:  10%|███▎                              |  ETA: 0:00:05


...converged after 108 iterations.
  1.866373 seconds (85.17 M allocations: 10.996 GiB, 15.76% gc time)


In [20]:
using BenchmarkTools

MISAEM.fit!(model, X_missing_fit, y_fit); # Warm-up

b1 = @benchmarkable MISAEM.fit!($model, $X_missing_fit, $y_fit)

tune!(b1);
run(b1)

SAEM Iterations:   9%|███                               |  ETA: 0:00:06


...converged after 108 iterations.


SAEM Iterations:  10%|███▍                              |  ETA: 0:00:05


...converged after 108 iterations.


SAEM Iterations:  10%|███▍                              |  ETA: 0:00:05

...

SAEM Iterations:   9%|███▏                              |  ETA: 0:00:05


...converged after 108 iterations.


SAEM Iterations:  10%|███▌                              |  ETA: 0:00:05


...converged after 108 iterations.


BenchmarkTools.Trial: 3 samples with 1 evaluation per sample.
 Range (min … max):  1.835 s …   1.923 s  ┊ GC (min … max): 15.41% … 17.10%
 Time  (median):     1.855 s              ┊ GC (median):    15.46%
 Time  (mean ± σ):   1.871 s ± 46.016 ms  ┊ GC (mean ± σ):  16.01% ±  0.96%

  █           █                                           █  
  █▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.84 s         Histogram: frequency by time        1.92 s <

 Memory estimate: 11.00 GiB, allocs estimate: 85171693.

In [21]:
using PyCall
# run(`$(PyCall.python) -m pip install numpy`)
# run(`$(PyCall.python) -m pip install misaem`)


misaem_py = pyimport("misaem")
SAEMLR = misaem_py.SAEMLogisticRegression

model_py = SAEMLR(maxruns=1000, random_state=42, var_cal=true);

In [22]:
model_py.fit(X_missing_fit, y_fit) # Warm-up
b2 = @benchmarkable model_py.fit($X_missing_fit, $y_fit)
tune!(b2)
run(b2)

...converged after 106 iterations.
...converged after 106 iterations.
...converged after 106 iterations.
...converged after 106 iterations.
...converged after 106 iterations.


 10%|███▉                                  | 105/1000 [00:01<00:08, 104.22it/s]


BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 52.530 s (0.00% GC) to evaluate,
 with a memory estimate of 19.78 KiB, over 22 allocations.